In [199]:
%load_ext kedro

The kedro extension is already loaded. To reload it, use:
  %reload_ext kedro


In [200]:
df = catalog.load('raw_data')
df.info()

[10/15/25 00:57:55] INFO     Loading data from raw_data (ParquetDataset)...                    ]8;id=806271;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=506255;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55043 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        55043 non-null  object
 1   category    55043 non-null  object
 2   item        55043 non-null  object
 3   unit        55043 non-null  object
 4   low_price   55043 non-null  object
 5   high_price  55043 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [201]:
import pandas as pd

In [202]:
df['date'] = pd.to_datetime(df['date'])

In [203]:

numeric_cols = ['low_price', 'high_price']

for col in numeric_cols:
    print(f"\n=== CHECKING {col} ===")
    
    numeric_series = pd.to_numeric(df[col], errors='coerce')
    non_numeric_mask = numeric_series.isna() & df[col].notna()
    
    if non_numeric_mask.any():
        print(f"Non-numeric values found in {col}:")
        non_numeric_values = df.loc[non_numeric_mask, col].unique()
        print(non_numeric_values)
        print(f"Count: {non_numeric_mask.sum()}")
    else:
        print(f"All values in {col} are numeric!")



=== CHECKING low_price ===
Non-numeric values found in low_price:
['বাজারে পাওয়া']
Count: 1

=== CHECKING high_price ===
Non-numeric values found in high_price:
['যায়নি']
Count: 1


In [204]:
df.loc[df['low_price'] == 'বাজারে পাওয়া', 'low_price']


42669    বাজারে পাওয়া
Name: low_price, dtype: object

In [205]:
df.loc[df['high_price'] == 'যায়নি', 'high_price']


42669    যায়নি
Name: high_price, dtype: object

**there seems to be one row with non numeric column. possible typo**

In [206]:
df['low_price'] = pd.to_numeric(df['low_price'], errors='coerce')
df['high_price'] = pd.to_numeric(df['high_price'], errors='coerce')

In [207]:
df['unit'].value_counts()


unit
প্রতি কেজি          38330
১ কেজি               4144
প্রতি লিটার          3106
প্রতি কেজি প্যাঃ     2072
প্রতি মেঃটন          2072
৫ লিটার              1502
১ লিটার              1368
প্রতি দিস্তা         1036
প্রতি হালি           1036
2 লিটার               375
50.00-52.00             2
Name: count, dtype: int64

In [208]:
df[df['unit'] == '50.00-52.00']

,date,category,item,unit,low_price,high_price
5664,2023-11-13,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0
41850,2023-11-15,মসলাঃ,তেজপাতা,50.00-52.00,150.0,200.0


**some units seem to have been mislabeled. ahve to fill them with proper unit for the item.**

In [209]:
df[df['item'] == 'তেজপাতা']['unit']


36       প্রতি কেজি
91       প্রতি কেজি
146      প্রতি কেজি
198      প্রতি কেজি
255      প্রতি কেজি
            ...    
54811    প্রতি কেজি
54865    প্রতি কেজি
54917    প্রতি কেজি
54971    প্রতি কেজি
55025    প্রতি কেজি
Name: unit, Length: 1036, dtype: object

In [210]:
df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'

                    WARNING  /tmp/ipykernel_18497/905726389.py:1: SettingWithCopyWarning:           ]8;id=50507;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=320332;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py#110\110]8;;\
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df[df['unit'] == '50.00-52.00']['unit'] = 'প্রতি কেজি'                                  
                                                                                                                   

In [211]:
unit_mapping = {
    'প্রতি কেজি': '1 KG',
    '১ কেজি': '1 KG', 
    'প্রতি লিটার': '1 Liter',
    'প্রতি কেজি প্যাঃ': '1 KG',
    'প্রতি মেঃটন': '1 Metric Tonne',
    '৫ লিটার': '5 Liter',
    '১ লিটার': '1 Liter',
    'প্রতি দিস্তা': '25 Pieces',
    'প্রতি হালি': '4 Pieces',
    '2 লিটার': '2 Liter'
}

df['unit'] = df['unit'].map(unit_mapping)

In [212]:
df['unit'].value_counts()


unit
1 KG              44546
1 Liter            4474
1 Metric Tonne     2072
5 Liter            1502
4 Pieces           1036
25 Pieces          1036
2 Liter             375
Name: count, dtype: int64

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55043 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        55043 non-null  datetime64[ns]
 1   category    55043 non-null  object        
 2   item        55043 non-null  object        
 3   unit        55041 non-null  object        
 4   low_price   55042 non-null  float64       
 5   high_price  55042 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 2.5+ MB


In [214]:
df[df['unit'].isna()]

,date,category,item,unit,low_price,high_price
5664,2023-11-13,মসলাঃ,তেজপাতা,NaN,150.0,200.0
41850,2023-11-15,মসলাঃ,তেজপাতা,NaN,150.0,200.0


In [215]:
df['category'].value_counts()


category
মসলাঃ                   16576
ডাল                      7252
বিবিধঃ                   7252
ভোজ্য তেল                6351
মাছ ও গোশত:              6216
আটা/ময়দা                 4144
গুড়া দুধ(প্যাকেটজাত)     4144
চাল                      3108
Name: count, dtype: int64

In [216]:
category_mapping = {
    'মসলাঃ': 'Spices',
    'ডাল': 'Lentils',
    'বিবিধঃ': 'Miscellaneous', 
    'ভোজ্য তেল': 'Edible Oil',
    'মাছ ও গোশত:': 'Fish & Meat',
    'আটা/ময়দা': 'Flour',
    'গুড়া দুধ(প্যাকেটজাত)': 'Powdered Milk (Packaged)',
    'চাল': 'Rice'
}

df['category'] = df['category'].map(category_mapping)

In [217]:
df

,date,category,item,unit,low_price,high_price
0,2025-08-01,Rice,চাল সরু (নাজির/মিনিকেট),1 KG,75.0,85.0
1,2025-08-01,Rice,চাল (মাঝারী)পাইজাম/আটাশ,1 KG,60.0,75.0
2,2025-08-01,Rice,চাল (মোটা)/স্বর্ণা/চায়না ইরি,1 KG,55.0,60.0
3,2025-08-01,Flour,আটা সাদা (খোলা),1 KG,40.0,45.0
4,2025-08-01,Flour,আটা (প্যাকেট),1 KG,50.0,55.0
...,...,...,...,...,...,...
55038,2024-07-08,Miscellaneous,লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে),1 KG,38.0,42.0
55039,2024-07-08,Miscellaneous,ডিম (ফার্ম),4 Pieces,45.0,50.0
55040,2024-07-08,Miscellaneous,লেখার কাগজ(সাদা),25 Pieces,30.0,35.0
55041,2024-07-08,Miscellaneous,"এম,এস রড (৬০ গ্রেড)",1 Metric Tonne,88500.0,89500.0


In [218]:
df['item'].value_counts().head(69)


item
সয়াবিন তেল (বোতল)               2410
চাল (মোটা)/স্বর্ণা/চায়না ইরি    1036
আটা সাদা (খোলা)                 1036
আটা (প্যাকেট)                   1036
চাল সরু (নাজির/মিনিকেট)         1036
                                ... 
রসুন(দেশী)                        60
আলু (নতুন/পুরাতন)(মানভেদে)        41
রসুন(দেশী) পুরাতন                 29
রসুন(দেশী) নতুন/পুরাতন            29
পিঁয়াজ (নতুন/পুরাতন) (দেশী)        7
Name: count, Length: 69, dtype: int64

In [219]:
for item in df['item'].unique():
    print(item)

চাল সরু (নাজির/মিনিকেট)
চাল (মাঝারী)পাইজাম/আটাশ
চাল (মোটা)/স্বর্ণা/চায়না ইরি
আটা সাদা (খোলা)
আটা (প্যাকেট)
ময়দা (খোলা)
ময়দা (প্যাকেট)
সয়াবিন তেল (লুজ)
সয়াবিন তেল (বোতল)
পাম অয়েল (লুজ)
সুপার পাম অয়েল (লুজ)
রাইস ব্রান তেল (বোতল)
মশুর ডাল (বড় দানা)
মশুর ডাল (মাঝারী দানা)
মশুর ডাল (ছোট দানা)
মুগ ডাল (মানভেদে)
এ্যাংকর ডাল
ছোলা (মানভেদে)
আলু (মানভেদে)
পিঁয়াজ (দেশী)
পিঁয়াজ (আমদানি)
রসুন (দেশী)
রসুন (আমদানি)
শুকনা মরিচ (দেশী)
শুকনা মরিচ (আমদানি)
হলুদ (দেশী)
হলুদ (আমদানি)
আদা (দেশী)
আদা (আমদানি)
জিরা
দারুচিনি
লবঙ্গ
এলাচ(ছোট)
ধনে
তেজপাতা
রুই
ইলিশ
গরু
খাসী
মুরগী(ব্রয়লার)
মুরগী (দেশী)
ডানো
ডিপ্লোমা (নিউজিল্যান্ড)
ফ্রেশ
মার্কস
চিনি
খেজুর(সাধারণ মানের)
লবণ(প্যাঃ)আয়োডিনযুক্ত
ডিম (ফার্ম)
লেখার কাগজ(সাদা)
এম,এস রড (৬০ গ্রেড)
এম,এস রড( ৪০ গ্রেড)
পাম অয়েল (সুপার)
মশূর ডাল (মাঝারী দানা)
লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)
চাল (মাঝারী)পাইজাম/লতা
রসুন(দেশী) নতুন/পুরাতন)
পাম অলিন (বোতল)
পিঁয়াজ (নতুন) (দেশী)
রসুন(দেশী)
আলু (নতুন, মানভেদে)
রসুন(দেশী) নতুন
আদা (দেশী)(নতুন)
আদা (দেশী) নতুন
আলু (নতুন/পুরাতন)(মানভেদে)

**There seems to be some redundent names that need merging**

In [220]:
pd.set_option('display.max_rows', 69)
item_counts = df['item'].value_counts()
print(item_counts)
pd.reset_option('display.max_rows')

item
সয়াবিন তেল (বোতল)                 2410
চাল (মোটা)/স্বর্ণা/চায়না ইরি      1036
আটা সাদা (খোলা)                   1036
আটা (প্যাকেট)                     1036
চাল সরু (নাজির/মিনিকেট)           1036
চিনি                              1036
ময়দা (খোলা)                       1036
ময়দা (প্যাকেট)                    1036
পাম অয়েল (লুজ)                    1036
মশুর ডাল (বড় দানা)                1036
এ্যাংকর ডাল                       1036
মুগ ডাল (মানভেদে)                 1036
মশুর ডাল (ছোট দানা)               1036
পিঁয়াজ (আমদানি)                   1036
ছোলা (মানভেদে)                    1036
হলুদ (আমদানি)                     1036
আদা (আমদানি)                      1036
হলুদ (দেশী)                       1036
শুকনা মরিচ (আমদানি)               1036
শুকনা মরিচ (দেশী)                 1036
রসুন (আমদানি)                     1036
ধনে                               1036
তেজপাতা                           1036
লবঙ্গ                             1036
এলাচ(ছোট)                        1036
রুই                 

In [221]:
def analyze_merge_possibilities_exact_dates(df):
    
    item_groups = {
        'rice': ['চাল (মাঝারী)পাইজাম/লতা', 'চাল (মাঝারী)পাইজাম/আটাশ'],
        'garlic': ['রসুন(দেশী) নতুন/পুরাতন)', 'রসুন(দেশী) নতুন', 'রসুন (দেশী) নতুন/পুরাতন', 'রসুন(দেশী) নতুন/পুরাতন', 'রসুন(দেশী) পুরাতন'],
        'ginger': ['আদা (দেশী)(নতুন)', 'আদা (দেশী) নতুন'],
        'onion': ['পিঁয়াজ (নতুন) (দেশী)', 'পিঁয়াজ (নতুন/পুরাতন) (দেশী)'],
        'potato': ['আলু (নতুন, মানভেদে)', 'আলু (নতুন/পুরাতন)(মানভেদে)'],
        'lentils': ['মশুর ডাল (মাঝারী দানা)'],
        'salt': ['লবণ(প্যাঃ)আয়োডিনযুক্ত'],
        'palm_oil': ['সুপার পাম অয়েল (লুজ)', 'পাম অলিন (বোতল)']
    }
    
    merge_suggestions = {}
    
    for group_name, items in item_groups.items():
        print(f"\n{'='*60}")
        print(f"ANALYZING MERGE POSSIBILITIES: {group_name.upper()}")
        print(f"{'='*60}")
        
        date_sets = {}
        for item in items:
            if item in df['item'].values:
                unique_dates = set(df[df['item'] == item]['date'].dt.date)
                date_sets[item] = unique_dates
                print(f"{item}: {len(unique_dates)} unique dates from {min(unique_dates)} to {max(unique_dates)}")
        
        items_list = list(date_sets.keys())
        possible_merges = []
        
        for i in range(len(items_list)):
            for j in range(i + 1, len(items_list)):
                item1, item2 = items_list[i], items_list[j]
                set1 = date_sets[item1]
                set2 = date_sets[item2]
                
                are_disjoint = set1.isdisjoint(set2)
                
                if are_disjoint:
                    possible_merges.append((item1, item2))
                    print(f"✅ CAN MERGE: {item1} + {item2} (completely disjoint dates)")
                else:
                    overlap_dates = set1.intersection(set2)
                    print(f"❌ CANNOT MERGE: {item1} + {item2} (shares {len(overlap_dates)} dates: {sorted(overlap_dates)[:3]}...)")
        
        merge_suggestions[group_name] = possible_merges
    
    return merge_suggestions

merge_analysis = analyze_merge_possibilities_exact_dates(df)


ANALYZING MERGE POSSIBILITIES: RICE
চাল (মাঝারী)পাইজাম/লতা: 700 unique dates from 2021-05-17 to 2024-10-29
চাল (মাঝারী)পাইজাম/আটাশ: 256 unique dates from 2024-10-30 to 2025-10-13
✅ CAN MERGE: চাল (মাঝারী)পাইজাম/লতা + চাল (মাঝারী)পাইজাম/আটাশ (completely disjoint dates)

ANALYZING MERGE POSSIBILITIES: GARLIC
রসুন(দেশী) নতুন/পুরাতন): 290 unique dates from 2022-02-18 to 2024-01-01
রসুন(দেশী) নতুন: 87 unique dates from 2024-03-04 to 2024-06-23
রসুন (দেশী) নতুন/পুরাতন: 73 unique dates from 2021-05-17 to 2021-08-27
রসুন(দেশী) নতুন/পুরাতন: 29 unique dates from 2024-02-01 to 2024-03-03
রসুন(দেশী) পুরাতন: 27 unique dates from 2024-01-02 to 2024-01-31
✅ CAN MERGE: রসুন(দেশী) নতুন/পুরাতন) + রসুন(দেশী) নতুন (completely disjoint dates)
✅ CAN MERGE: রসুন(দেশী) নতুন/পুরাতন) + রসুন (দেশী) নতুন/পুরাতন (completely disjoint dates)
✅ CAN MERGE: রসুন(দেশী) নতুন/পুরাতন) + রসুন(দেশী) নতুন/পুরাতন (completely disjoint dates)
✅ CAN MERGE: রসুন(দেশী) নতুন/পুরাতন) + রসুন(দেশী) পুরাতন (completely disjoint dates)
✅

In [222]:
def safe_merge_items(df):
    safe_merge_rules = {
        'চাল (মাঝারী)পাইজাম/আটাশ': 'চাল (মাঝারী)পাইজাম/লতা',
        
        'রসুন(দেশী) নতুন': 'রসুন(দেশী) নতুন/পুরাতন)',
        'রসুন (দেশী) নতুন/পুরাতন': 'রসুন(দেশী) নতুন/পুরাতন)',
        'রসুন(দেশী) নতুন/পুরাতন': 'রসুন(দেশী) নতুন/পুরাতন)',
        'রসুন(দেশী) পুরাতন': 'রসুন(দেশী) নতুন/পুরাতন)',
        
        'আদা (দেশী) নতুন': 'আদা (দেশী)(নতুন)',
        
        'পিঁয়াজ (নতুন/পুরাতন) (দেশী)': 'পিঁয়াজ (নতুন) (দেশী)',
        
        'আলু (নতুন/পুরাতন)(মানভেদে)': 'আলু (নতুন, মানভেদে)',
        
        'মশুর ডাল (মাঝারী দানা)': 'মশূর ডাল (মাঝারী দানা)',
        'লবণ(প্যাঃ)আয়োডিনযুক্ত': 'লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)',
    }
    
    df['item'] = df['item'].replace(safe_merge_rules)
    
    print("Safe merging completed!")
    print(f"Items merged: {len(safe_merge_rules)}")
    print(f"Unique items after safe merging: {len(df['item'].unique())}")
    
    return df

df = safe_merge_items(df)

Safe merging completed!
Items merged: 10
Unique items after safe merging: 59


In [223]:
cnt = df['item'].value_counts()
cnt


item
সয়াবিন তেল (বোতল)                 2410
চাল (মাঝারী)পাইজাম/লতা            1036
চাল সরু (নাজির/মিনিকেট)           1036
চাল (মোটা)/স্বর্ণা/চায়না ইরি      1036
আটা সাদা (খোলা)                   1036
ময়দা (খোলা)                       1036
আটা (প্যাকেট)                     1036
ময়দা (প্যাকেট)                    1036
পাম অয়েল (লুজ)                    1036
মশূর ডাল (মাঝারী দানা)            1036
মশুর ডাল (বড় দানা)                1036
হলুদ (দেশী)                       1036
হলুদ (আমদানি)                     1036
মশুর ডাল (ছোট দানা)               1036
মুগ ডাল (মানভেদে)                 1036
ছোলা (মানভেদে)                    1036
এ্যাংকর ডাল                       1036
রসুন (আমদানি)                     1036
পিঁয়াজ (আমদানি)                   1036
শুকনা মরিচ (আমদানি)               1036
শুকনা মরিচ (দেশী)                 1036
এলাচ(ছোট)                        1036
ধনে                               1036
লবঙ্গ                             1036
দারুচিনি                          1036
জিরা               